# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
spaceship.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [4]:
spaceship.dropna (inplace=True)

In [5]:
spaceship.isnull().sum()

PassengerId     0
HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Name            0
Transported     0
dtype: int64

In [6]:
spaceship= spaceship.drop(columns=["Name", "PassengerId", "Cabin"])

In [7]:
X = spaceship.drop(columns="Transported")
y = spaceship["Transported"]

In [8]:
X_num = X.select_dtypes(include='number')
X_cat = X.select_dtypes(include='object')

X_num

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0.0,0.0,0.0,0.0,0.0
1,24.0,109.0,9.0,25.0,549.0,44.0
2,58.0,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0
8689,18.0,0.0,0.0,0.0,0.0,0.0
8690,26.0,0.0,0.0,1872.0,1.0,0.0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0


In [9]:
X_cat

,HomePlanet,CryoSleep,Destination,VIP
0,Europa,False,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,False,TRAPPIST-1e,True
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False
...,...,...,...,...
8688,Europa,False,55 Cancri e,True
8689,Earth,True,PSO J318.5-22,False
8690,Earth,False,TRAPPIST-1e,False
8691,Europa,False,55 Cancri e,False


In [10]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder().fit(X_cat) 
encoded = encoder.transform(X_cat).toarray()
unique_values = [encoder.categories_[i] for i in range(len(X_cat.columns))]
onehot_encoded = pd.DataFrame(encoded, columns=[f"{col}_{value}" for col, values in zip(X_cat.columns, unique_values) for value in values]) 
onehot_encoded

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
6601,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6602,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6603,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6604,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0


In [11]:
onehot_encoded.reset_index (drop=True, inplace= True)
X_num.reset_index (drop=True, inplace= True)

In [12]:
X = pd.concat([X_num, onehot_encoded], axis=1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [16]:
transformer = MinMaxScaler().fit(X_train) #Aplicar el min y el max, pero recuerda que X  tiene que ser valores numericos
X_train_normalized_array = transformer.transform(X_train) #Convertirlo entre 0 y 1 
X_train_norm= pd.DataFrame(X_train_normalized_array, columns=X_train.columns)

In [17]:
X_test_normalized_array = transformer.transform(X_test) #Convertirlo entre 0 y 1 
X_test_norm = pd.DataFrame(X_test_normalized_array, columns=X_test.columns) 

X_test_norm

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CryoSleep_False,CryoSleep_True,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP_False,VIP_True
0,0.632911,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,0.227848,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.189873,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,0.658228,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.784810,0.000000,0.054775,0.00000,0.077740,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,0.240506,0.000000,0.000000,0.05468,0.000045,0.001672,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1318,0.468354,0.030242,0.115185,0.00000,0.000045,0.008409,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1319,0.544304,0.000202,0.178748,0.00000,0.000312,0.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1320,0.177215,0.000000,0.000000,0.00000,0.000000,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


Now perform the same as before:
- Feature Scaling
- Feature Selection


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [18]:
bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=20),
                               n_estimators=100,
                               max_samples = 1000)

In [19]:
bagging_reg.fit(X_train_norm, y_train)

BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=20),
                 max_samples=1000, n_estimators=100)

- Evaluate your model

In [20]:
pred = bagging_reg.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test_norm, y_test))

MAE 0.28602622746096473
RMSE 0.3828864065684499
R2 score 0.41359199866039864


**Grid/Random Search**

For this lab we will use Grid Search.

In [21]:
from sklearn.model_selection import GridSearchCV # type: ignore
from sklearn.model_selection import RandomizedSearchCV # type: ignore

- Define hyperparameters to fine tune.

In [ ]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [ ]:
ada_reg = AdaBoostRegressor(DecisionTreeRegressor())


- Run Grid Search

In [ ]:
model = GridSearchCV(estimator = ada_reg, param_grid = grid, cv=5)

In [ ]:
model.fit(X_train_norm, y_train)

In [ ]:
model.best_params_

In [ ]:
best_model = model.best_estimator_

- Evaluate your model

In [ ]:
pred = best_model.predict(X_test_norm)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", best_model.score(X_test_norm, y_test))